![](https://www.thoughtco.com/thmb/SxrROMIc0e0iynH4BHluBqiQYI0=/3052x1934/filters:fill(auto,1)/R.M.S.Titanic-5baae8d6c9e77c0025e53f86.jpg)

Hi folks. We are going to do an EDA alonside of data preparation such as missing data and hopfully looking forward to get a good accuracy by training our model with Logistic Regression.

# Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Read the test data
# YOU CAN SKIP THIS CELL
#THIS PART IS FOR COMPETITION. TEST DATA PREPRATION THAT WILL USE AGAIN FOR OUR TRAIN DATA LATER
test = pd.read_csv('../input/titanic/test.csv')
test = test.drop(['Name', 'Ticket', 'Cabin'], axis = 1)

for i in range(len(test)):
    if(test.loc[i,'Sex'] == 'male'):
        test.loc[i,'Sex'] = 1;
    else:
        test.loc[i,'Sex'] = 0
        
new_embark = pd.get_dummies(test['Embarked'],drop_first=True)
test.drop(['Embarked'],axis=1,inplace=True)
test = pd.concat([test,new_embark],axis=1)
test['Fare'] = test['Fare'].fillna(test['Fare'].median())

test['Age'] = test["Age"].fillna(test['Age'].median())
test2 = test
test.head()

# EDA

In [ ]:
df = pd.read_csv("../input/titanic/train.csv")
df.info()

In [ ]:
df.head(10)

In [ ]:
df.describe()

In [ ]:
print(df['Survived'].value_counts())
sns.countplot(data = df, x = df['Survived'],palette='RdBu_r')

In [ ]:
sns.countplot(data = df, x = df['SibSp'], palette='RdBu_r')

In [ ]:
sns.boxplot(x='Pclass',y='Fare',data=df)

In [ ]:
sns.boxplot(x='Embarked',y='Age',data=df, palette='RdBu_r')

In [ ]:
sns.countplot(x='Survived',data=df,hue='Pclass', palette='RdBu_r')

In [ ]:
df = df.drop(['Name', 'Ticket', 'Cabin'],axis = 1)

In [ ]:
sns.pairplot(df,hue='Survived')

# Converting Categorical Features

Now we should convert our categorical features to dummy data or any numerical data that make sense. Otherwise our model can't work with them. Since out categories are not too much we can easily use dummy variable. I convert the sex feature by my self and convert the embarked feature by pandas later.

In [ ]:
for i in range(len(df)):
    if(df.loc[i,'Sex'] == 'male'):
        df.loc[i,'Sex'] = 1;
    else:
        df.loc[i,'Sex'] = 0
        
df.head()

# Missing data

In [ ]:
df.isnull().sum()

Roughly 20 percent of the Age data is missing. The proportion of Age missing is likely small enough for reasonable replacement with some form of imputation

we have 177 missing in Age feature and 2 in Embarked. Obviously we could find out those 177 ages but a reasonable way to fill them out is to put Age averege in those field. Since we don't know where those two passanger embark the ship we can either put the most common embarkation or drop both rows.

In [ ]:
df['Age'] = df["Age"].fillna(df['Age'].median())
df[df['Embarked'].isnull()]

In [ ]:
df = df.drop([61,829], axis = 0)

In [ ]:
df.isnull().sum()

In [ ]:
df['Embarked'].value_counts()

In [ ]:
new_embark = pd.get_dummies(df['Embarked'],drop_first=True)
df.drop(['Embarked'],axis=1,inplace=True)
df = pd.concat([df,new_embark],axis=1)
df.head(10)

In [ ]:
sns.heatmap(df.corr(), annot=True)

ok by checking the heat map we can understand that there isn't any hughly corrolated feauter in out data and also we don't have any redundant feature.

# Train-Test Split

In [ ]:
from sklearn.model_selection import train_test_split
y = df['Survived']
X = df.drop('Survived', axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler= StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
test = scaler.transform(test)

# Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression 

log_model = LogisticRegression()
log_model.fit(X_train,y_train)
y_pred = log_model.predict(X_test)

In [ ]:
param = log_model.get_params()
pd.DataFrame(data = param, index = [0])

Also we can use LogisticRegressionCV to use built-in corss validation.

# Model Evaluation

Metrics for evaluating a classification model are:
* Accuracy
* Recall(a.k.a. Sensitivity)
* Precision
* F1-Score: combination of recall and precision

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, plot_confusion_matrix
accuracy_score(y_test, y_pred)

In [ ]:
from sklearn.metrics import plot_confusion_matrix
plot_confusion_matrix(log_model, X_train, y_train)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
y_test_pred= log_model.predict(test)
test

In [ ]:
my_submission = pd.DataFrame({'PassengerId': test2.PassengerId, 'Survived': y_test_pred})
# you could use any filename. We choose submission here
my_submission.to_csv('submission4.csv', index=False)